In [1]:
#!/usr/bin/env python3
"""
YOLOv8 Fine-tuning Script for Enemy Detection
"""

from ultralytics import YOLO
import os

In [2]:
def train_enemy_detection_model():
    """
    Fine-tune YOLOv8 model for enemy detection using augmented data
    """
    
    # Configuration
    DATA_YAML = "augmented_data/dataset.yaml"
    PRETRAINED_MODEL = "yolov8n.pt"  # or yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
    PROJECT_NAME = "enemy_detection"
    EXPERIMENT_NAME = "run1"
    
    # Training parameters
    EPOCHS = 100
    BATCH_SIZE = 16  # Adjust based on your GPU memory
    IMAGE_SIZE = 640
    
    print("=" * 60)
    print("YOLOv8 Enemy Detection Fine-tuning")
    print("=" * 60)
    
    # Check if dataset exists
    if not os.path.exists(DATA_YAML):
        print(f"❌ Dataset file not found: {DATA_YAML}")
        print("Please run the data augmentation notebook first!")
        return
    
    # Load pretrained model
    print(f"📦 Loading pretrained model: {PRETRAINED_MODEL}")
    model = YOLO(PRETRAINED_MODEL)
    
    # Display model info
    print(f"📊 Model architecture: {PRETRAINED_MODEL}")
    print(f"📁 Dataset: {DATA_YAML}")
    print(f"🎯 Target: Enemy detection (1 class)")
    print(f"⚙️  Epochs: {EPOCHS}")
    print(f"📏 Image size: {IMAGE_SIZE}")
    print(f"🔢 Batch size: {BATCH_SIZE}")
    
    print("\n🚀 Starting training...")
    
    # Train the model
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=EXPERIMENT_NAME,
        save=True,
        save_period=10,  # Save checkpoint every 10 epochs
        patience=50,     # Early stopping patience
        device=0,        # Use GPU 0, change to 'cpu' if no GPU
        workers=8,       # Number of worker threads
        cache=True,      # Cache images for faster training
        # Data augmentation (YOLOv8 handles this automatically)
        hsv_h=0.015,     # Image HSV-Hue augmentation
        hsv_s=0.7,       # Image HSV-Saturation augmentation
        hsv_v=0.4,       # Image HSV-Value augmentation
        degrees=0.0,     # Image rotation (+/- deg)
        translate=0.1,   # Image translation (+/- fraction)
        scale=0.5,       # Image scale (+/- gain)
        shear=0.0,       # Image shear (+/- deg)
        perspective=0.0, # Image perspective (+/- fraction)
        flipud=0.0,      # Image flip up-down (probability)
        fliplr=0.5,      # Image flip left-right (probability)
        mosaic=1.0,      # Image mosaic (probability)
        mixup=0.0,       # Image mixup (probability)
    )
    
    print("\n✅ Training completed!")
    
    # Print results
    print(f"📁 Results saved to: {PROJECT_NAME}/{EXPERIMENT_NAME}")
    print(f"🏆 Best model: {PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt")
    print(f"📈 Training plots: {PROJECT_NAME}/{EXPERIMENT_NAME}/")
    
    # Validate the model
    print("\n🔍 Running validation...")
    metrics = model.val()
    
    print("\n📊 Validation Metrics:")
    print(f"   mAP50: {metrics.box.map50:.3f}")
    print(f"   mAP50-95: {metrics.box.map:.3f}")
    print(f"   Precision: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")
    
    return model, results


In [3]:
def train_person_detection_model():
    """
    Fine-tune YOLOv8 model to detect game characters as 'person' class
    This leverages pretrained knowledge for faster, better training
    """
    
    # Configuration - Use existing augmented data
    DATA_YAML = "augmented_data/dataset.yaml"  # Your existing data
    PRETRAINED_MODEL = "yolov8n.pt"  
    PROJECT_NAME = "person_detection"
    EXPERIMENT_NAME = "game_characters"
    
    # Training parameters (much fewer epochs needed!)
    EPOCHS = 25  # Reduced from 100 since we're fine-tuning existing person class
    BATCH_SIZE = 16
    IMAGE_SIZE = 640
    
    print("=" * 60)
    print("YOLOv8 Person Detection Fine-tuning (Game Characters)")
    print("=" * 60)
    
    # Check if dataset exists
    if not os.path.exists(DATA_YAML):
        print(f"❌ Dataset file not found: {DATA_YAML}")
        print("Please run the data augmentation notebook first!")
        return
    
    # Load pretrained model
    print(f"📦 Loading pretrained model: {PRETRAINED_MODEL}")
    model = YOLO(PRETRAINED_MODEL)
    
    # Display model info
    print(f"📊 Model architecture: {PRETRAINED_MODEL}")
    print(f"📁 Dataset: {DATA_YAML}")
    print(f"🎯 Target: Person detection (leveraging pretrained class)")
    print(f"⚙️  Epochs: {EPOCHS} (reduced due to transfer learning)")
    print(f"📏 Image size: {IMAGE_SIZE}")
    print(f"🔢 Batch size: {BATCH_SIZE}")
    print(f"💡 Strategy: Game characters → Person class (ID 0)")
    
    print("\n🚀 Starting fine-tuning...")
    
    # Train the model with transfer learning settings
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=EXPERIMENT_NAME,
        save=True,
        save_period=5,   # Save more frequently
        patience=15,     # Earlier stopping
        device=0,        # Use GPU 0, 'cpu' if no GPU
        workers=8,
        cache=True,
        # Optimized settings for fine-tuning person detection
        lr0=0.001,       # Lower initial learning rate for fine-tuning
        lrf=0.1,         # Learning rate final multiplier
        warmup_epochs=3, # Warmup epochs
        # Conservative augmentation for stability
        hsv_h=0.01,      # Minimal hue shift
        hsv_s=0.4,       # Moderate saturation
        hsv_v=0.3,       # Moderate value changes
        degrees=5.0,     # Small rotation
        translate=0.05,  # Small translation
        scale=0.2,       # Small scale changes
        fliplr=0.5,      # Horizontal flip OK
        mosaic=0.8,      # Reduced mosaic
        mixup=0.1,       # Light mixup
    )
    
    print("\n✅ Fine-tuning completed!")
    print(f"📁 Results saved to: {PROJECT_NAME}/{EXPERIMENT_NAME}")
    print(f"🏆 Best model: {PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt")
    
    # Validate
    print("\n🔍 Running validation...")
    metrics = model.val()
    
    print("\n📊 Validation Metrics:")
    print(f"   mAP50: {metrics.box.map50:.3f}")
    print(f"   mAP50-95: {metrics.box.map:.3f}")
    print(f"   Precision: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")
    
    return model, results

In [4]:
import shutil
import random
from pathlib import Path
import json

def split_dataset(train_ratio=0.8, val_ratio=0.2):
    """
    Split the augmented dataset into training and validation sets
    
    Args:
        train_ratio: Ratio of data for training (0.8 = 80%)
        val_ratio: Ratio of data for validation (0.2 = 20%)
    """
    
    print("📊 Splitting dataset into train/validation sets...")
    
    # Paths
    data_dir = Path("augmented_data")
    images_dir = data_dir
    labels_dir = data_dir / "labels"
    
    # Create train/val directories
    train_images_dir = data_dir / "train" / "images"
    train_labels_dir = data_dir / "train" / "labels"
    val_images_dir = data_dir / "val" / "images"
    val_labels_dir = data_dir / "val" / "labels"
    
    # Create directories
    for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Get all image files
    image_files = list(images_dir.glob("*.jpg")) + list(images_dir.glob("*.png"))
    image_files = [f for f in image_files if f.is_file() and not f.parent.name in ['train', 'val']]
    
    print(f"Found {len(image_files)} images to split")
    
    if len(image_files) == 0:
        print("❌ No images found! Make sure you've run the data augmentation first.")
        return False
    
    # Shuffle the files
    random.shuffle(image_files)
    
    # Calculate split indices
    train_count = int(len(image_files) * train_ratio)
    
    train_files = image_files[:train_count]
    val_files = image_files[train_count:]
    
    print(f"📁 Training set: {len(train_files)} images ({len(train_files)/len(image_files)*100:.1f}%)")
    print(f"📁 Validation set: {len(val_files)} images ({len(val_files)/len(image_files)*100:.1f}%)")
    
    # Copy files to train directory
    for img_file in train_files:
        # Copy image
        shutil.copy2(img_file, train_images_dir / img_file.name)
        
        # Copy corresponding label
        label_file = labels_dir / (img_file.stem + ".txt")
        if label_file.exists():
            shutil.copy2(label_file, train_labels_dir / label_file.name)
    
    # Copy files to validation directory
    for img_file in val_files:
        # Copy image
        shutil.copy2(img_file, val_images_dir / img_file.name)
        
        # Copy corresponding label
        label_file = labels_dir / (img_file.stem + ".txt")
        if label_file.exists():
            shutil.copy2(label_file, val_labels_dir / label_file.name)
    
    # Create updated dataset.yaml
    dataset_yaml_content = f"""path: {data_dir.absolute()}
train: train/images
val: val/images

nc: 1
names:
  0: person
"""
    
    with open(data_dir / "dataset_split.yaml", 'w') as f:
        f.write(dataset_yaml_content)
    
    print("✅ Dataset split completed!")
    print(f"📝 Created dataset_split.yaml with train/val paths")
    
    return True

def train_person_detection_model():
    """
    Fine-tune YOLOv8 model to detect game characters as 'person' class
    This leverages pretrained knowledge for faster, better training
    """
    
    # First, split the dataset
    print("🔄 Preparing dataset...")
    if not split_dataset(train_ratio=0.8, val_ratio=0.2):
        return None, None
    
    # Configuration - Use split dataset
    DATA_YAML = "augmented_data/dataset_split.yaml"  # Use split dataset
    PRETRAINED_MODEL = "yolov8n.pt"  
    PROJECT_NAME = "person_detection"
    EXPERIMENT_NAME = "game_characters"
    
    # Training parameters (much fewer epochs needed!)
    EPOCHS = 25  # Reduced from 100 since we're fine-tuning existing person class
    BATCH_SIZE = 16
    IMAGE_SIZE = 640
    
    print("\n" + "=" * 60)
    print("YOLOv8 Person Detection Fine-tuning (Game Characters)")
    print("=" * 60)
    
    # Check if dataset exists
    if not os.path.exists(DATA_YAML):
        print(f"❌ Dataset file not found: {DATA_YAML}")
        print("Please run the data augmentation notebook first!")
        return None, None
    
    # Load pretrained model
    print(f"📦 Loading pretrained model: {PRETRAINED_MODEL}")
    model = YOLO(PRETRAINED_MODEL)
    
    # Display model info
    print(f"📊 Model architecture: {PRETRAINED_MODEL}")
    print(f"📁 Dataset: {DATA_YAML}")
    print(f"🎯 Target: Person detection (leveraging pretrained class)")
    print(f"⚙️  Epochs: {EPOCHS} (reduced due to transfer learning)")
    print(f"📏 Image size: {IMAGE_SIZE}")
    print(f"🔢 Batch size: {BATCH_SIZE}")
    print(f"💡 Strategy: Game characters → Person class (ID 0)")
    print(f"📊 Data split: 80% train, 20% validation")
    
    print("\n🚀 Starting fine-tuning...")
    
    # Train the model with transfer learning settings
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=EXPERIMENT_NAME,
        save=True,
        save_period=5,   # Save more frequently
        patience=15,     # Earlier stopping
        device=0,        # Use GPU 0, 'cpu' if no GPU
        workers=8,
        cache=True,
        # Optimized settings for fine-tuning person detection
        lr0=0.001,       # Lower initial learning rate for fine-tuning
        lrf=0.1,         # Learning rate final multiplier
        warmup_epochs=3, # Warmup epochs
        # Conservative augmentation for stability
        hsv_h=0.01,      # Minimal hue shift
        hsv_s=0.4,       # Moderate saturation
        hsv_v=0.3,       # Moderate value changes
        degrees=5.0,     # Small rotation
        translate=0.05,  # Small translation
        scale=0.2,       # Small scale changes
        fliplr=0.5,      # Horizontal flip OK
        mosaic=0.8,      # Reduced mosaic
        mixup=0.1,       # Light mixup
    )
    
    print("\n✅ Fine-tuning completed!")
    print(f"📁 Results saved to: {PROJECT_NAME}/{EXPERIMENT_NAME}")
    print(f"🏆 Best model: {PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt")
    
    # Validate on the validation set
    print("\n🔍 Running validation on held-out validation set...")
    metrics = model.val()
    
    print("\n📊 Final Validation Metrics:")
    print(f"   mAP50: {metrics.box.map50:.3f}")
    print(f"   mAP50-95: {metrics.box.map:.3f}")
    print(f"   Precision: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")
    
    return model, results

In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from datetime import datetime
import pandas as pd

def save_detection_visualizations(model, output_dir="detection_outputs", max_images=20):
    """
    Save images with bounding boxes drawn around detected characters
    
    Args:
        model: Trained YOLO model
        output_dir: Directory to save visualization images
        max_images: Maximum number of images to process
    """
    
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"💾 Saving detection visualizations to: {output_dir}")
    print("-" * 50)
    
    # Get validation images
    val_images_dir = Path("augmented_data/val/images")
    if not val_images_dir.exists():
        print("❌ No validation images found! Make sure you've split the dataset.")
        return
    
    val_images = list(val_images_dir.glob("*.jpg"))[:max_images]
    
    if len(val_images) == 0:
        print("❌ No validation images found!")
        return
    
    detection_summary = []
    
    for i, img_path in enumerate(val_images):
        print(f"Processing image {i+1}/{len(val_images)}: {img_path.name}")
        
        # Run inference
        results = model(str(img_path), conf=0.3, save=False, verbose=False)
        
        # Load original image
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        original_height, original_width = img_rgb.shape[:2]
        
        # Draw detections
        detections_count = 0
        high_conf_count = 0
        
        for r in results:
            boxes = r.boxes
            if boxes is not None:
                for box in boxes:
                    # Get box coordinates and confidence
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    conf = box.conf[0].cpu().numpy()
                    
                    detections_count += 1
                    if conf > 0.7:
                        high_conf_count += 1
                    
                    # Choose color based on confidence
                    if conf > 0.8:
                        color = (0, 255, 0)  # Bright green for very high confidence
                        thickness = 3
                    elif conf > 0.5:
                        color = (255, 165, 0)  # Orange for medium confidence
                        thickness = 2
                    else:
                        color = (255, 255, 0)  # Yellow for low confidence
                        thickness = 2
                    
                    # Draw bounding box
                    cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
                    
                    # Calculate box size for text scaling
                    box_width = x2 - x1
                    font_scale = min(max(box_width / 200, 0.4), 1.0)  # Scale font with box size
                    
                    # Add confidence label
                    label = f"Person {conf:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)[0]
                    
                    # Draw label background
                    cv2.rectangle(img_rgb, 
                                (int(x1), int(y1) - label_size[1] - 10), 
                                (int(x1) + label_size[0], int(y1)), 
                                color, -1)
                    
                    # Draw label text
                    cv2.putText(img_rgb, label, (int(x1), int(y1) - 5), 
                              cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), 2)
        
        # Add image info text at top
        info_text = f"Detections: {detections_count} | High Conf (>0.7): {high_conf_count}"
        cv2.putText(img_rgb, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(img_rgb, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 1)
        
        # Save the image
        output_filename = f"detection_{i+1:03d}_{img_path.stem}_conf{detections_count}.jpg"
        output_file_path = output_path / output_filename
        
        # Convert back to BGR for saving
        img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(output_file_path), img_bgr)
        
        # Store summary info
        detection_summary.append({
            'image': img_path.name,
            'total_detections': detections_count,
            'high_confidence': high_conf_count,
            'output_file': output_filename
        })
    
    # Create summary report
    summary_path = output_path / "detection_summary.txt"
    with open(summary_path, 'w') as f:
        f.write("DETECTION VISUALIZATION SUMMARY\n")
        f.write("=" * 40 + "\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total images processed: {len(val_images)}\n")
        f.write(f"Output directory: {output_dir}\n\n")
        
        f.write("COLOR CODING:\n")
        f.write("🟢 Bright Green: High confidence (>0.8)\n")
        f.write("🟠 Orange: Medium confidence (0.5-0.8)\n")
        f.write("🟡 Yellow: Low confidence (<0.5)\n\n")
        
        f.write("IMAGE DETAILS:\n")
        f.write("-" * 40 + "\n")
        
        total_detections = 0
        total_high_conf = 0
        
        for item in detection_summary:
            f.write(f"File: {item['output_file']}\n")
            f.write(f"  Original: {item['image']}\n")
            f.write(f"  Total detections: {item['total_detections']}\n")
            f.write(f"  High confidence: {item['high_confidence']}\n")
            f.write(f"  Detection rate: {item['total_detections']/1:.1f} per image\n\n")
            
            total_detections += item['total_detections']
            total_high_conf += item['high_confidence']
        
        f.write("OVERALL STATISTICS:\n")
        f.write("-" * 40 + "\n")
        f.write(f"Average detections per image: {total_detections/len(val_images):.1f}\n")
        f.write(f"Average high-confidence per image: {total_high_conf/len(val_images):.1f}\n")
        f.write(f"High-confidence rate: {(total_high_conf/total_detections*100) if total_detections > 0 else 0:.1f}%\n")
    
    print(f"\n✅ Saved {len(val_images)} visualization images")
    print(f"📄 Summary report: {summary_path}")
    print(f"🎨 Color coding: Green (high conf) | Orange (medium) | Yellow (low)")
    
    return detection_summary

def comprehensive_model_evaluation():
    """
    Comprehensive evaluation of the trained model with detailed metrics and visualizations
    """
    
    # Load the best trained model
    model_path = "person_detection/game_characters/weights/best.pt"
    
    if not os.path.exists(model_path):
        print(f"❌ Trained model not found: {model_path}")
        print("Please run training first!")
        return
    
    print("🧪 Starting comprehensive model evaluation...")
    print("=" * 60)
    
    model = YOLO(model_path)
    
    # 1. Save detection visualizations FIRST
    print("🎨 1. SAVING DETECTION VISUALIZATIONS")
    print("-" * 40)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    detection_output_dir = f"detection_outputs_{timestamp}"
    
    detection_summary = save_detection_visualizations(model, detection_output_dir, max_images=25)
    
    # 2. Detailed validation metrics
    print(f"\n📊 2. DETAILED VALIDATION METRICS")
    print("-" * 40)
    metrics = model.val()
    
    # Create detailed metrics report
    metrics_report = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'Precision': metrics.box.mp,
        'Recall': metrics.box.mr,
        'F1-Score': 2 * (metrics.box.mp * metrics.box.mr) / (metrics.box.mp + metrics.box.mr) if (metrics.box.mp + metrics.box.mr) > 0 else 0
    }
    
    print(f"🎯 mAP@0.5: {metrics_report['mAP50']:.3f}")
    print(f"🎯 mAP@0.5:0.95: {metrics_report['mAP50-95']:.3f}")
    print(f"🎯 Precision: {metrics_report['Precision']:.3f}")
    print(f"🎯 Recall: {metrics_report['Recall']:.3f}")
    print(f"🎯 F1-Score: {metrics_report['F1-Score']:.3f}")
    
    # 3. Display sample visualizations in notebook
    print(f"\n🖼️  3. SAMPLE VISUALIZATIONS IN NOTEBOOK")
    print("-" * 40)
    
    val_images_dir = Path("augmented_data/val/images")
    if val_images_dir.exists():
        val_images = list(val_images_dir.glob("*.jpg"))[:6]  # Show 6 images in notebook
        
        if len(val_images) > 0:
            fig, axes = plt.subplots(2, 3, figsize=(18, 12))
            axes = axes.flatten()
            
            detection_stats = {'total_detections': 0, 'high_conf_detections': 0}
            
            for i, img_path in enumerate(val_images):
                # Run inference
                results = model(str(img_path), conf=0.3)  # Lower confidence for testing
                
                # Load and process image
                img = cv2.imread(str(img_path))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Draw detections
                for r in results:
                    boxes = r.boxes
                    if boxes is not None:
                        for box in boxes:
                            # Get box coordinates
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            conf = box.conf[0].cpu().numpy()
                            
                            detection_stats['total_detections'] += 1
                            if conf > 0.7:
                                detection_stats['high_conf_detections'] += 1
                            
                            # Draw rectangle
                            color = (0, 255, 0) if conf > 0.7 else (255, 165, 0)  # Green for high conf, orange for low
                            cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                            
                            # Add label
                            label = f"Person {conf:.2f}"
                            cv2.putText(img_rgb, label, (int(x1), int(y1)-10), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                # Display
                axes[i].imshow(img_rgb)
                axes[i].set_title(f"Val Image {i+1}\nDetections: {len(results[0].boxes) if results[0].boxes is not None else 0}")
                axes[i].axis('off')
            
            plt.tight_layout()
            plt.suptitle("Model Performance on Validation Set\n(Green: High Confidence >0.7, Orange: Low Confidence)", y=1.02)
            plt.show()
            
            print(f"📈 Detection Statistics (notebook samples):")
            print(f"   Total detections: {detection_stats['total_detections']}")
            print(f"   High confidence (>0.7): {detection_stats['high_conf_detections']}")
            print(f"   Detection rate: {detection_stats['high_conf_detections']/len(val_images):.1f} per image")
        
        else:
            print("❌ No validation images found!")
    
    # 4. Confidence threshold analysis
    print(f"\n📊 4. CONFIDENCE THRESHOLD ANALYSIS")
    print("-" * 40)
    
    if val_images_dir.exists():
        thresholds = [0.3, 0.5, 0.7, 0.9]
        threshold_results = {}
        
        test_images = list(val_images_dir.glob("*.jpg"))[:10]  # Test on 10 images
        
        for thresh in thresholds:
            total_detections = 0
            for img_path in test_images:
                results = model(str(img_path), conf=thresh)
                for r in results:
                    if r.boxes is not None:
                        total_detections += len(r.boxes)
            
            avg_detections = total_detections / len(test_images)
            threshold_results[thresh] = avg_detections
            print(f"   Confidence {thresh}: {avg_detections:.1f} avg detections per image")
    
    # 5. Performance timing test
    print(f"\n⏱️  5. PERFORMANCE TIMING TEST")
    print("-" * 40)
    
    if len(val_images) > 0:
        import time
        
        # Warm up
        _ = model(str(val_images[0]))
        
        # Time multiple inferences
        times = []
        for i in range(10):
            start_time = time.time()
            _ = model(str(val_images[i % len(val_images)]))
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        fps = 1.0 / avg_time
        
        print(f"🚀 Average inference time: {avg_time*1000:.1f}ms")
        print(f"🚀 Estimated FPS: {fps:.1f}")
        print(f"🚀 Real-time capable: {'✅ Yes' if fps >= 30 else '❌ No (for 30+ FPS)'}")
    
    # 6. Save comprehensive report
    print(f"\n💾 6. SAVING EVALUATION REPORT")
    print("-" * 40)
    
    # Create detailed report
    report = {
        'timestamp': timestamp,
        'model_path': model_path,
        'detection_output_dir': detection_output_dir,
        'metrics': metrics_report,
        'timing': {
            'avg_inference_time_ms': avg_time * 1000 if 'avg_time' in locals() else 'N/A',
            'estimated_fps': fps if 'fps' in locals() else 'N/A'
        },
        'detection_stats': detection_stats if 'detection_stats' in locals() else {},
        'threshold_analysis': threshold_results if 'threshold_results' in locals() else {},
        'detection_summary': detection_summary if 'detection_summary' in locals() else {}
    }
    
    # Save as JSON
    report_path = f"person_detection/evaluation_report_{timestamp}.json"
    with open(report_path, 'w') as f:
        json.dump(report, f, indent=2, default=str)
    
    # 7. Create CLEAN METRICS FILE
    print(f"\n📊 7. CREATING CLEAN METRICS SUMMARY")
    print("-" * 40)
    
    # Calculate additional metrics
    total_val_images = len(list(val_images_dir.glob("*.jpg"))) if val_images_dir.exists() else 0
    total_detections = sum(item['total_detections'] for item in detection_summary) if detection_summary else 0
    total_high_conf = sum(item['high_confidence'] for item in detection_summary) if detection_summary else 0
    
    # Create clean metrics file
    clean_metrics_path = f"person_detection/METRICS_SUMMARY_{timestamp}.txt"
    with open(clean_metrics_path, 'w') as f:
        f.write("=" * 60 + "\n")
        f.write("CV AIM ASSIST - MODEL PERFORMANCE METRICS\n")
        f.write("=" * 60 + "\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Model: {model_path}\n")
        f.write("=" * 60 + "\n\n")
        
        # Core Performance Metrics
        f.write("🎯 CORE PERFORMANCE METRICS\n")
        f.write("-" * 30 + "\n")
        f.write(f"mAP@0.5:           {metrics_report['mAP50']:.3f}     (Higher = Better, Max = 1.0)\n")
        f.write(f"mAP@0.5:0.95:      {metrics_report['mAP50-95']:.3f}     (Higher = Better, Max = 1.0)\n")
        f.write(f"Precision:         {metrics_report['Precision']:.3f}     (Higher = Better, Max = 1.0)\n")
        f.write(f"Recall:            {metrics_report['Recall']:.3f}     (Higher = Better, Max = 1.0)\n")
        f.write(f"F1-Score:          {metrics_report['F1-Score']:.3f}     (Higher = Better, Max = 1.0)\n\n")
        
        # Performance Rating
        overall_score = (metrics_report['mAP50'] + metrics_report['Precision'] + metrics_report['Recall']) / 3
        if overall_score >= 0.8:
            rating = "EXCELLENT ⭐⭐⭐⭐⭐"
        elif overall_score >= 0.7:
            rating = "VERY GOOD ⭐⭐⭐⭐"
        elif overall_score >= 0.6:
            rating = "GOOD ⭐⭐⭐"
        elif overall_score >= 0.5:
            rating = "FAIR ⭐⭐"
        else:
            rating = "NEEDS IMPROVEMENT ⭐"
        
        f.write(f"📈 OVERALL RATING\n")
        f.write("-" * 30 + "\n")
        f.write(f"Overall Score:     {overall_score:.3f}     {rating}\n\n")
        
        # Speed Performance
        if 'avg_time' in locals():
            f.write(f"⚡ SPEED PERFORMANCE\n")
            f.write("-" * 30 + "\n")
            f.write(f"Inference Time:    {avg_time*1000:.1f} ms        (Lower = Better)\n")
            f.write(f"Estimated FPS:     {fps:.1f}            (Higher = Better)\n")
            real_time_status = "YES ✅" if fps >= 30 else "NO ❌"
            f.write(f"Real-time Ready:   {real_time_status}        (30+ FPS needed)\n\n")
        
        # Detection Analysis
        if detection_summary:
            avg_detections = total_detections / len(detection_summary)
            avg_high_conf = total_high_conf / len(detection_summary)
            high_conf_rate = (total_high_conf / total_detections * 100) if total_detections > 0 else 0
            
            f.write(f"🔍 DETECTION ANALYSIS\n")
            f.write("-" * 30 + "\n")
            f.write(f"Images Tested:     {len(detection_summary)}\n")
            f.write(f"Total Detections:  {total_detections}\n")
            f.write(f"High Confidence:   {total_high_conf}            (>0.7 threshold)\n")
            f.write(f"Avg per Image:     {avg_detections:.1f}            (detections)\n")
            f.write(f"High Conf Rate:    {high_conf_rate:.1f}%           (reliability)\n\n")
        
        # Confidence Thresholds
        if 'threshold_results' in locals():
            f.write(f"🎚️  CONFIDENCE THRESHOLDS\n")
            f.write("-" * 30 + "\n")
            for thresh, detections in threshold_results.items():
                f.write(f"Threshold {thresh}:     {detections:.1f} avg detections\n")
            f.write("\n")
        
        # Model Quality Assessment
        f.write(f"📋 MODEL QUALITY ASSESSMENT\n")
        f.write("-" * 30 + "\n")
        
        # Precision assessment
        if metrics_report['Precision'] >= 0.8:
            f.write("Precision:         EXCELLENT - Very few false positives\n")
        elif metrics_report['Precision'] >= 0.7:
            f.write("Precision:         GOOD - Some false positives\n")
        elif metrics_report['Precision'] >= 0.6:
            f.write("Precision:         FAIR - Moderate false positives\n")
        else:
            f.write("Precision:         POOR - Too many false positives\n")
        
        # Recall assessment
        if metrics_report['Recall'] >= 0.8:
            f.write("Recall:            EXCELLENT - Finds most characters\n")
        elif metrics_report['Recall'] >= 0.7:
            f.write("Recall:            GOOD - Finds many characters\n")
        elif metrics_report['Recall'] >= 0.6:
            f.write("Recall:            FAIR - Misses some characters\n")
        else:
            f.write("Recall:            POOR - Misses too many characters\n")
        
        # mAP assessment
        if metrics_report['mAP50'] >= 0.8:
            f.write("mAP@0.5:           EXCELLENT - Very accurate bounding boxes\n")
        elif metrics_report['mAP50'] >= 0.7:
            f.write("mAP@0.5:           GOOD - Accurate bounding boxes\n")
        elif metrics_report['mAP50'] >= 0.6:
            f.write("mAP@0.5:           FAIR - Somewhat accurate boxes\n")
        else:
            f.write("mAP@0.5:           POOR - Inaccurate bounding boxes\n")
        
        f.write("\n")
        
        # Recommendations
        f.write(f"💡 RECOMMENDATIONS\n")
        f.write("-" * 30 + "\n")
        
        if overall_score >= 0.7:
            f.write("✅ Model is ready for production use!\n")
            f.write("✅ Good balance of precision and recall\n")
            if 'fps' in locals() and fps >= 30:
                f.write("✅ Fast enough for real-time gaming\n")
            else:
                f.write("⚠️  Consider using smaller model for better speed\n")
        elif overall_score >= 0.6:
            f.write("⚠️  Model is decent but could be improved\n")
            if metrics_report['Precision'] < 0.7:
                f.write("📈 Consider: More training data to reduce false positives\n")
            if metrics_report['Recall'] < 0.7:
                f.write("📈 Consider: Lower confidence threshold or more training\n")
        else:
            f.write("❌ Model needs significant improvement\n")
            f.write("📈 Suggested actions:\n")
            f.write("   - Collect more diverse training data\n")
            f.write("   - Train for more epochs\n")
            f.write("   - Check data quality and labeling\n")
        
        f.write("\n")
        f.write("=" * 60 + "\n")
        f.write("For detailed analysis, check the visual outputs in:\n")
        f.write(f"{detection_output_dir}/\n")
        f.write("=" * 60 + "\n")
    
    # Save as readable text (detailed version)
    txt_report_path = f"person_detection/evaluation_report_{timestamp}.txt"
    with open(txt_report_path, 'w') as f:
        f.write("CV AIM ASSIST MODEL EVALUATION REPORT\n")
        f.write("=" * 50 + "\n")
        f.write(f"Generated: {timestamp}\n")
        f.write(f"Model: {model_path}\n")
        f.write(f"Detection Visualizations: {detection_output_dir}/\n\n")
        
        f.write("VALIDATION METRICS:\n")
        f.write("-" * 20 + "\n")
        for key, value in metrics_report.items():
            f.write(f"{key}: {value:.3f}\n")
        
        f.write(f"\nPERFORMANCE:\n")
        f.write("-" * 20 + "\n")
        if 'avg_time' in locals():
            f.write(f"Inference Time: {avg_time*1000:.1f}ms\n")
            f.write(f"Estimated FPS: {fps:.1f}\n")
            f.write(f"Real-time Ready: {'Yes' if fps >= 30 else 'No'}\n")
        
        if 'detection_stats' in locals():
            f.write(f"\nDETECTION ANALYSIS:\n")
            f.write("-" * 20 + "\n")
            f.write(f"Total Detections: {detection_stats['total_detections']}\n")
            f.write(f"High Confidence: {detection_stats['high_conf_detections']}\n")
        
        if 'threshold_results' in locals():
            f.write(f"\nCONFIDENCE THRESHOLDS:\n")
            f.write("-" * 20 + "\n")
            for thresh, detections in threshold_results.items():
                f.write(f"Threshold {thresh}: {detections:.1f} avg detections\n")
    
    print(f"📄 JSON report saved: {report_path}")
    print(f"📄 Detailed report saved: {txt_report_path}")
    print(f"📊 CLEAN METRICS saved: {clean_metrics_path}")
    print(f"🎨 Visual outputs saved: {detection_output_dir}/")
    
    print(f"\n🎉 EVALUATION COMPLETE!")
    print("=" * 60)
    
    # Return summary for further use
    return {
        'model': model,
        'metrics': metrics_report,
        'report_path': report_path,
        'detection_output_dir': detection_output_dir
    }

In [ ]:
# 🎯 EXECUTE: Complete Training Pipeline with Evaluation

print("🚀 COMPLETE TRAINING PIPELINE")
print("=" * 60)
print("📋 Pipeline Steps:")
print("   1. Split data into train/validation (80%/20%)")
print("   2. Fine-tune YOLO person detection")
print("   3. Comprehensive model evaluation")
print("   4. Save detailed performance report")
print("=" * 60)

# Step 1 & 2: Train the model (includes data splitting)
try:
    print("🔥 STEP 1-2: Training with data splitting...")
    model, results = train_person_detection_model()
    
    if model is not None:
        print("\n" + "🎉" * 20)
        print("TRAINING SUCCESS!")
        print("🎉" * 20)
        
        # Step 3: Comprehensive evaluation
        print(f"\n🔍 STEP 3: Comprehensive Model Evaluation...")
        print("This will test the model on validation data and save detailed reports")
        
        evaluation_results = comprehensive_model_evaluation()
        
        if evaluation_results:
            print("\n" + "✅" * 20)
            print("EVALUATION COMPLETE!")
            print("✅" * 20)
            
            print(f"\n📊 FINAL PERFORMANCE SUMMARY:")
            print("-" * 40)
            metrics = evaluation_results['metrics']
            print(f"🎯 mAP@0.5: {metrics['mAP50']:.3f}")
            print(f"🎯 Precision: {metrics['Precision']:.3f}")
            print(f"🎯 Recall: {metrics['Recall']:.3f}")
            print(f"🎯 F1-Score: {metrics['F1-Score']:.3f}")
            
            print(f"\n📁 Files Generated:")
            print(f"   🏆 Best Model: person_detection/game_characters/weights/best.pt")
            print(f"   📊 Training Plots: person_detection/game_characters/")
            print(f"   � Evaluation Report: {evaluation_results['report_path']}")
            
            print(f"\n🎮 READY FOR AIM ASSIST!")
            print("Your model is trained and evaluated. Key takeaways:")
            print("✅ Detects game characters as 'person' class")
            print("✅ Leverages pretrained human detection knowledge")
            print("✅ Fast inference for real-time gaming")
            print("✅ Detailed performance metrics saved")
            
        else:
            print("❌ Evaluation failed - but training was successful!")
            print("💡 You can still use the trained model")
    
    else:
        print("❌ Training failed!")
        print("💡 Make sure you've run the data augmentation notebook first")
        
except Exception as e:
    print(f"❌ Pipeline failed: {e}")
    print("💡 Check that you've run data augmentation and have enough data")

print("\n" + "="*60)
print("🎯 NEXT STEPS:")
print("1. Review the evaluation report for detailed metrics")
print("2. Check training plots in person_detection/game_characters/")
print("3. Use best.pt model for real-time aim assist")
print("4. Test the model on new game footage")
print("="*60)

🚀 COMPLETE TRAINING PIPELINE
📋 Pipeline Steps:
   1. Split data into train/validation (80%/20%)
   2. Fine-tune YOLO person detection
   3. Comprehensive model evaluation
   4. Save detailed performance report
🔥 STEP 1-2: Training with data splitting...
🔄 Preparing dataset...
📊 Splitting dataset into train/validation sets...
Found 7670 images to split
📁 Training set: 6136 images (80.0%)
📁 Validation set: 1534 images (20.0%)
✅ Dataset split completed!
📝 Created dataset_split.yaml with train/val paths

YOLOv8 Person Detection Fine-tuning (Game Characters)
📦 Loading pretrained model: yolov8n.pt
📊 Model architecture: yolov8n.pt
📁 Dataset: augmented_data/dataset_split.yaml
🎯 Target: Person detection (leveraging pretrained class)
⚙️  Epochs: 25 (reduced due to transfer learning)
📏 Image size: 640
🔢 Batch size: 16
💡 Strategy: Game characters → Person class (ID 0)
📊 Data split: 80% train, 20% validation

🚀 Starting fine-tuning...
New https://pypi.org/project/ultralytics/8.3.184 available 😃 Upd

train: Scanning /home/kronbii/repos/CV-aim-assist/augmented_data/train/labels... 6136 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6136/6136 [00:02<00:00, 2327.54it/s]


train: New cache created: /home/kronbii/repos/CV-aim-assist/augmented_data/train/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (3.9GB RAM): 100%|██████████| 6136/6136 [00:08<00:00, 720.82it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1688.0±1161.3 MB/s, size: 271.9 KB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:06<00:00,  7.16it/s]


                   all       1534       3764      0.999          1      0.995      0.897
Speed: 0.2ms preprocess, 0.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to person_detection/game_characters

✅ Fine-tuning completed!
📁 Results saved to: person_detection/game_characters
🏆 Best model: person_detection/game_characters/weights/best.pt

🔍 Running validation on held-out validation set...
Ultralytics 8.3.179 🚀 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7815MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 347.4±55.5 MB/s, size: 267.8 KB)


val: Scanning /home/kronbii/repos/CV-aim-assist/augmented_data/val/labels.cache... 1534 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1534/1534 [00:00<?, ?it/s]

WARNING ⚠️ val: 1.5GB RAM required to cache images with 50% safety margin but only 1.1/15.5GB available, not caching images



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   2%|▏         | 2/96 [00:01<01:04,  1.45it/s]